# Ranking Models :

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install --upgrade python-terrier
!pip install --upgrade git+https://github.com/Georgetown-IR-Lab/OpenNIR
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_t5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.9/337.9 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.7 MB/s eta 0:00:00
  Preparing metadata (setu

In [4]:
import pyterrier as pt
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

terrier-assemblies 5.9 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done
terrier-prf -SNAPSHOT jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8



In [5]:
import onir_pt


Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


# Ranking

## TREC:

### Index de TREC :

In [6]:
import zipfile
import os

# Remplacez 'path_to_your_zip_file' par le chemin de votre fichier zip sur Google Drive
zip_path = '/content/drive/MyDrive/RI/index.zip'
extract_path = '/content/drive/MyDrive/RI/index'

# Créez le répertoire de destination s'il n'existe pas
os.makedirs(extract_path, exist_ok=True)

# Décompressez le fichier zip
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Vérifiez que le contenu a été extrait
print("Contenu du répertoire extrait : ", os.listdir(extract_path))


Contenu du répertoire extrait :  ['antique', 'trec']


In [7]:

# Spécifiez le chemin vers l'index décompressé
trec_index_path = extract_path+'/trec'

# Chargez l'index
index_trec = pt.IndexFactory.of(trec_index_path)

# Testez le chargement de l'index
print(index_trec.getCollectionStatistics().toString())

Number of documents: 8841823
Number of terms: 1170682
Number of postings: 215238456
Number of fields: 1
Number of tokens: 288759529
Field names: [text]
Positions:   false



### Le dataset en question :

In [41]:
dataset_trec= pt.datasets.get_dataset('irds:msmarco-passage/trec-dl-2019/judged')

## Retrieving documents :

In [ ]:
model_to_use = "BM25"
model = None
trec_data = True
limit =
if model_to_use == "BM25":
        model = pt.BatchRetrieve(index_trec, wmodel="BM25") % limit
elif model_to_use == "BM25+RM3":
    bm_25 = pt.BatchRetrieve(index_trec, wmodel="BM25") % limit
    model = bm_25 >> pt.rewrite.RM3(index_trec) >> bm_25


### Éxperimentations :

In [9]:
# Chemin vers le fichier CSV
import pandas as pd
import re
chemin_fichier = "variations_trec2019_labeled.csv"

# Lecture du fichier CSV

donnee2 = pd.read_csv(chemin_fichier)

donnee2["query"] = donnee2.apply(lambda r, re=re: re.sub('[\W_]', ' ',  r['original_query'].lower()), axis=1)
donnee2["variation"] = donnee2.apply(lambda r, re=re: re.sub('[\W_]', ' ',  r['variation'].lower()), axis=1)
donnee2["variation"] = donnee2.apply(lambda r: r['query'] if r['variation'].strip() == "" else r['variation'], axis=1)
donnee2['qid'] = donnee2['q_id']

# donnee2 = donnee2[['q_id','original_query']]
# donnee2.columns = ['qid','query']


In [ ]:
# # Créer le modèle BM25 pour la récupération initiale
# bm25_initial = pt.BatchRetrieve(index_trec, wmodel="BM25", verbose=True) % limit

# # Créer le modèle RM3 pour la reformulation des requêtes
# rm3 = pt.rewrite.RM3(index_trec)

# # Créer le modèle BM25 pour la récupération avec les requêtes reformulées
# bm25_followup = pt.BatchRetrieve(index_trec, wmodel="BM25", verbose=True)

# # Créer le pipeline combiné
# pipeline = bm25_initial >> rm3 >> bm25_followup

# results = pipeline.transform(donnee2.drop_duplicates())


# results_original_qs = model.transform(query_variations[['query','qid']].drop_duplicates())[['qid', 'docno', 'score']]

# results_original_qs.columns = [['qid', 'docno', 'score_original_query']]

# all_together = [df_scores_original_q]
# for indice_method, df in enumerate (result_per_method):
#     df_score_per_method = df[['qid', 'docno', 'score']]
#     df_score_per_method.columns = [['qid', 'docno', 'score_'+variante_sname[indice_method]]]
#     all_together.append(df_score_per_method)


#Pour afficher les correlatione nyre des ordonnancement de chaque model sur chaque methode
# cor_df = functools.reduce(lambda df1, df2: df1.merge(df2), all_together).corr(method='kendall')
# cor_df.to_csv("{}/query_corr_model_{}.csv".format(".", model_to_use))



#### BM25 :

In [ ]:
bm_25 = pt.BatchRetrieve(index_trec, wmodel="BM25") % limit

model_to_use = "BM25"
variante_sname = []
result_per_method = []

query_variations=donnee2
query_variations['qid'] = query_variations['q_id'].astype(str)
query_variations['query'] = query_variations['original_query']

for method in query_variations['method'].unique():
    print("Methode : ",method)
    query_variation = query_variations[query_variations['method'] == method]
    query_variation['query'] = query_variation['variation']

    result_per_method.append(bm_25.transform(query_variation[['qid', 'query']]))
    variante_sname.append("{}+{}".format(model_to_use, method))
    print("{}+{}".format(model_to_use, method))



Methode :  WordSwapNeighboringCharacterSwap


<ipython-input-20-e5f0e01201be>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_variation['query'] = query_variation['variation']


BM25+WordSwapNeighboringCharacterSwap
Methode :  WordSwapQWERTY


<ipython-input-20-e5f0e01201be>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_variation['query'] = query_variation['variation']


BM25+WordSwapQWERTY
Methode :  WordSwapRandomCharacterSubstitution


...

BM25+ramsrigouthamg/t5_paraphraser
Methode :  WordSwapWordNet


<ipython-input-20-e5f0e01201be>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_variation['query'] = query_variation['variation']


BM25+WordSwapWordNet
Methode :  WordSwapEmbedding


<ipython-input-20-e5f0e01201be>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_variation['query'] = query_variation['variation']


BM25+WordSwapEmbedding


In [ ]:
# Experimentations :
metrics = ['ndcg_cut_10']
df = pt.Experiment(
        [bm_25] + result_per_method,
        query_variations[['qid','query']].drop_duplicates(),
        dataset_trec.get_qrels(),
        metrics,
        baseline=0,
        names = [model_to_use]+variante_sname)




In [ ]:
df

,name,ndcg_cut_10,ndcg_cut_10 +,ndcg_cut_10 -,ndcg_cut_10 p-value
0,BM25,0.479540,NaN,NaN,NaN
1,BM25+WordSwapNeighboringCharacterSwap,0.274701,8.0,29.0,0.000131
2,BM25+WordSwapQWERTY,0.243548,5.0,33.0,0.000006
3,BM25+WordSwapRandomCharacterSubstitution,0.231450,7.0,31.0,0.000009
4,BM25+naturality_by_removing_stop_words,0.477754,1.0,2.0,0.551436
5,BM25+summarization_with_t5-base_from_descripti...,0.421481,9.0,19.0,0.051226
6,BM25+WordInnerSwapRandom,0.479540,0.0,0.0,NaN
7,BM25+back_translation_pivot_language_de,0.396437,4.0,14.0,0.023552
8,BM25+ramsrigouthamg/t5_paraphraser,0.472212,2.0,5.0,0.465643
9,BM25+WordSwapWordNet,0.348766,3.0,27.0,0.000721


In [ ]:
# Chemin où vous souhaitez enregistrer le fichier CSV dans Google Drive
file_path = '/content/drive/My Drive/RI/BM_25_original.csv'

# Enregistrer le DataFrame en tant que fichier CSV
df.to_csv(file_path, index=False)

In [ ]:

# Experimentations par requete :
metrics = ['ndcg_cut_10']
df_perq = pt.Experiment(
        [bm_25] + result_per_method,
        query_variations[['query','qid']].drop_duplicates(),
        dataset_trec.get_qrels(),
        metrics,
        perquery = True,
        names = [model_to_use]+variante_sname)

In [ ]:
df_perq

,name,qid,measure,value
0,BM25,1037798,ndcg_cut_10,0.152866
1,BM25,104861,ndcg_cut_10,0.776306
2,BM25,1063750,ndcg_cut_10,0.142811
3,BM25,1103812,ndcg_cut_10,0.507699
4,BM25,1106007,ndcg_cut_10,0.368773
...,...,...,...,...
252,BM25+summarization_with_t5-base_from_descripti...,855410,ndcg_cut_10,0.881199
253,BM25+summarization_with_t5-base_from_descripti...,87181,ndcg_cut_10,0.533127
254,BM25+summarization_with_t5-base_from_descripti...,87452,ndcg_cut_10,0.423192
255,BM25+summarization_with_t5-base_from_descripti...,915593,ndcg_cut_10,0.300943


In [ ]:
# Chemin où vous souhaitez enregistrer le fichier CSV dans Google Drive
file_path = '/content/drive/My Drive/RI/BM_25_original_q.csv'

# Enregistrer le DataFrame en tant que fichier CSV
df_perq.to_csv(file_path, index=False)

#### Méthodes : run experiment :

In [10]:
# Chemin vers le fichier CSV
import re
chemin_fichier = "variations_trec2019_labeled.csv"

# Lecture du fichier CSV

query_variations = pd.read_csv(chemin_fichier)

query_variations["query"] = query_variations.apply(lambda r, re=re: re.sub('[\W_]', ' ',  r['original_query'].lower()), axis=1)
query_variations["variation"] = query_variations.apply(lambda r, re=re: re.sub('[\W_]', ' ',  r['variation'].lower()), axis=1)
query_variations["variation"] = query_variations.apply(lambda r: r['query'] if r['variation'].strip() == "" else r['variation'], axis=1)
query_variations['qid'] = query_variations['q_id']

In [74]:
limit = 100
def run(model,model_name,query_variations):
  variante_sname = []
  result_per_method = []
  #pre traitement
  query_variations['qid'] = query_variations['q_id'].astype(str)
  query_variations['query'] = query_variations['original_query']

  for method in query_variations['method'].unique():
      print("Methode : ",method)
      query_variation = query_variations[query_variations['method'] == method]
      query_variation['query'] = query_variation['variation']

      result_per_method.append(model.transform(query_variation[['qid', 'query']]))
      variante_sname.append("{}+{}".format(model_name, method))
      print("{}+{}".format(model_name, method))

  return result_per_method,variante_sname



def experiment(model,result_per_method,query_variations,dataset_trec,variante_sname,model_to_use):
  metrics = ['ndcg_cut_10']
  df = pt.Experiment(
        [model] + result_per_method,
        query_variations[['qid','query']].drop_duplicates(),
        dataset_trec.get_qrels(),
        metrics,
        baseline=0,
        names = [model_to_use]+variante_sname)
  df_perq = pt.Experiment(
        [model] + result_per_method,
        query_variations[['query','qid']].drop_duplicates(),
        dataset_trec.get_qrels(),
        metrics,
        perquery = True,
        names = [model_to_use]+variante_sname)
  return df,df_perq


#### **BM25 BERT** :
*   ICI LE CODE DE BASE A D'ABORD CALCULER LE SCORE AVEC RECUPRATION DE 100 DOC LE FICHIER RI/BM25_BERT.csv et ENSUITE POUR COMPARER EST CE QU'IL Y AVAIT UNE DIFFÉRENCE ENTRE 10 OU 100 DOCUMENTS ON A RELANCER ICI POUR AVOIR LE FICHIER RI/BM25_BERT_10.csv

In [76]:
#rajouter limite!!! avec 10 ici

def BM25_BERT(dataset,index,limit):
      bert = onir_pt.reranker.from_checkpoint("https://macavaney.us/scibert-medmarco.tar.gz",expected_md5="854966d0b61543ffffa44cea627ab63b")
      model = pt.BatchRetrieve(index, wmodel="BM25") % limit >> pt.text.get_text(dataset, 'text') >> bert
      return model

BM25_et_BERT_1000 = BM25_BERT(dataset_trec,index_trec,1000)


[2024-05-19 12:14:08,324][onir_pt][INFO] using cached checkpoint: /root/data/onir/model_checkpoints/1153203ea2abbdf7f760fa6956a83d83


In [ ]:
result_per_method,variante_sname = run(BM25_et_BERT_1000,"BM25+BERT",query_variations)
df,df_q = experiment(BM25_et_BERT_1000,result_per_method,query_variations,dataset_trec,variante_sname,"BM25+BERT")
df

Methode :  WordSwapNeighboringCharacterSwap


<ipython-input-74-831b6c94f792>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_variation['query'] = query_variation['variation']


[2024-05-19 12:15:08,933][onir_pt][DEBUG] using GPU (deterministic)
[2024-05-19 12:15:09,198][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/9829 s<?, ?it/s]

[2024-05-19 12:20:31,049][onir_pt][DEBUG] [finished] batches: [05:22] [9829it] [30.54it/s]
BM25+BERT+WordSwapNeighboringCharacterSwap
Methode :  WordSwapQWERTY


...

<ipython-input-74-831b6c94f792>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_variation['query'] = query_variation['variation']


[2024-05-19 13:07:03,912][onir_pt][DEBUG] using GPU (deterministic)
[2024-05-19 13:07:03,916][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/10502 s<?, ?it/s]

[2024-05-19 13:12:50,131][onir_pt][DEBUG] [finished] batches: [05:46] [10502it] [30.33it/s]
BM25+BERT+WordSwapEmbedding
[2024-05-19 13:12:54,964][onir_pt][DEBUG] using GPU (deterministic)
[2024-05-19 13:12:54,968][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/10502 s<?, ?it/s]

In [ ]:
# Chemin où vous souhaitez enregistrer le fichier CSV dans Google Drive
file_path = '/content/drive/My Drive/RI/BM25_BERT_1000.csv'

# Enregistrer le DataFrame en tant que fichier CSV
df.to_csv(file_path, index=False)
# Chemin où vous souhaitez enregistrer le fichier CSV dans Google Drive
file_path = '/content/drive/My Drive/RI/BM25_BERT_1000_q.csv'

# Enregistrer le DataFrame en tant que fichier CSV
df_q.to_csv(file_path, index=False)

#### BM25 KNRM:

In [ ]:
def BM25_KNRM(dataset,index,limit):
      knrm = onir_pt.reranker.from_checkpoint('https://macavaney.us/knrm.medmarco.tar.gz',  expected_md5="d70b1d4f899690dae51161537e69ed5a")
      model = pt.BatchRetrieve(index, wmodel="BM25") % limit >> pt.text.get_text(dataset, 'text') >> knrm
      return model
model_bm25_knrm =  BM25_KNRM (dataset_trec,index_trec,limit)

[2024-05-19 09:09:44,459][onir.util.download][DEBUG] downloaded https://macavaney.us/knrm.medmarco.tar.gz s] [1.43k] [1.48MB/s] [md5 hash verified]
[2024-05-19 09:09:44,471][WordvecHashVocab][DEBUG] [starting] reading cached at /root/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p
[2024-05-19 09:09:57,773][WordvecHashVocab][DEBUG] [finished] reading cached at /root/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p [13.30s]


In [ ]:
result_per_method,variante_sname = run(model_bm25_knrm,"BM25+KNRM",query_variations)
df,df_q = experiment(model_bm25_knrm,result_per_method,query_variations,dataset_trec,variante_sname,"BM25+KNRM")
df

Methode :  WordSwapNeighboringCharacterSwap


<ipython-input-31-3849ac34831d>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_variation['query'] = query_variation['variation']


[2024-05-19 09:10:44,099][onir_pt][DEBUG] using GPU (deterministic)
[2024-05-19 09:10:44,529][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/1018 s<?, ?it/s]

[2024-05-19 09:10:46,982][onir_pt][DEBUG] [finished] batches: [2.45s] [1018it] [415.17it/s]
BM25+KNRM+WordSwapNeighboringCharacterSwap
Methode :  WordSwapQWERTY


...

batches:   0%|          | 0/1052 s<?, ?it/s]

[2024-05-19 09:11:43,088][onir_pt][DEBUG] [finished] batches: [2.77s] [1052it] [379.38it/s]
[2024-05-19 09:11:47,467][onir_pt][DEBUG] using GPU (deterministic)
[2024-05-19 09:11:47,469][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/1052 s<?, ?it/s]

[2024-05-19 09:11:49,152][onir_pt][DEBUG] [finished] batches: [1.68s] [1052it] [625.25it/s]


,name,ndcg_cut_10,ndcg_cut_10 +,ndcg_cut_10 -,ndcg_cut_10 p-value
0,BM25+KNRM,0.496959,NaN,NaN,NaN
1,BM25+KNRM+WordSwapNeighboringCharacterSwap,0.317241,11.0,29.0,1.021549e-03
2,BM25+KNRM+WordSwapQWERTY,0.275153,6.0,34.0,5.236292e-05
3,BM25+KNRM+WordSwapRandomCharacterSubstitution,0.222502,4.0,37.0,1.332003e-07
4,BM25+KNRM+naturality_by_removing_stop_words,0.489481,18.0,15.0,6.128722e-01
5,BM25+KNRM+summarization_with_t5-base_from_desc...,0.394968,13.0,27.0,2.363134e-02
6,BM25+KNRM+WordInnerSwapRandom,0.496959,0.0,0.0,NaN
7,BM25+KNRM+back_translation_pivot_language_de,0.376331,7.0,23.0,9.331250e-04
8,BM25+KNRM+ramsrigouthamg/t5_paraphraser,0.437429,11.0,17.0,1.245522e-02
9,BM25+KNRM+WordSwapWordNet,0.359142,14.0,25.0,5.976324e-04


In [ ]:
# Chemin où vous souhaitez enregistrer le fichier CSV dans Google Drive
file_path = '/content/drive/My Drive/RI/BM25_KNRM.csv'

# Enregistrer le DataFrame en tant que fichier CSV
df.to_csv(file_path, index=False)
# Chemin où vous souhaitez enregistrer le fichier CSV dans Google Drive
file_path = '/content/drive/My Drive/RI/BM25_KNRM_q.csv'

# Enregistrer le DataFrame en tant que fichier CSV
df_q.to_csv(file_path, index=False)

#### BM25_T5:

In [49]:
from pyterrier_t5 import MonoT5ReRanker

def BM25T5 (dataset,index,limit):
      monoT5 = MonoT5ReRanker()
      model = pt.BatchRetrieve(index, wmodel="BM25") % limit >> pt.text.get_text(dataset, 'text') >> monoT5
      return model
model_bm25_T5 =  BM25T5 (dataset_trec,index_trec,limit)

In [ ]:
result_per_method,variante_sname = run(model_bm25_T5,"BM25+T5",query_variations)
df,df_q = experiment(model_bm25_T5,result_per_method,query_variations,dataset_trec,variante_sname,"BM25+T5")
df

Methode :  WordSwapNeighboringCharacterSwap


<ipython-input-31-3849ac34831d>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_variation['query'] = query_variation['variation']


monoT5:   0%|          | 0/1018 s<?, ?batches/s]

BM25+T5+WordSwapNeighboringCharacterSwap
Methode :  WordSwapQWERTY


<ipython-input-31-3849ac34831d>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_variation['query'] = query_variation['variation']


...

monoT5:   0%|          | 0/1052 s<?, ?batches/s]

BM25+T5+WordSwapEmbedding


monoT5:   0%|          | 0/1052 s<?, ?batches/s]

monoT5:   0%|          | 0/1052 s<?, ?batches/s]

,name,ndcg_cut_10,ndcg_cut_10 +,ndcg_cut_10 -,ndcg_cut_10 p-value
0,BM25+T5,0.699780,NaN,NaN,NaN
1,BM25+T5+WordSwapNeighboringCharacterSwap,0.494417,6.0,31.0,0.000064
2,BM25+T5+WordSwapQWERTY,0.446060,10.0,29.0,0.000017
3,BM25+T5+WordSwapRandomCharacterSubstitution,0.396311,10.0,30.0,0.000002
4,BM25+T5+naturality_by_removing_stop_words,0.686159,13.0,19.0,0.049199
5,BM25+T5+summarization_with_t5-base_from_descri...,0.571683,13.0,26.0,0.000707
6,BM25+T5+WordInnerSwapRandom,0.697020,18.0,15.0,0.742609
7,BM25+T5+back_translation_pivot_language_de,0.605816,9.0,16.0,0.009197
8,BM25+T5+ramsrigouthamg/t5_paraphraser,0.704520,12.0,11.0,0.559038
9,BM25+T5+WordSwapWordNet,0.545733,9.0,28.0,0.000154


In [ ]:
# Chemin où vous souhaitez enregistrer le fichier CSV dans Google Drive
file_path = '/content/drive/My Drive/RI/BM25_T5.csv'

# Enregistrer le DataFrame en tant que fichier CSV
df.to_csv(file_path, index=False)
# Chemin où vous souhaitez enregistrer le fichier CSV dans Google Drive
file_path = '/content/drive/My Drive/RI/BM25_T5_q.csv'

# Enregistrer le DataFrame en tant que fichier CSV
df_q.to_csv(file_path, index=False)

####  BM25 EPIC:

In [65]:

def EPIC(dataset,index,limit):
  config = {'batch_size': 4}
  reranker = onir_pt.reranker.from_checkpoint("https://macavaney.us/pt_models/msmarco.epic.seed42.tar.gz", config=config)
  model = pt.BatchRetrieve(index, wmodel="BM25") % limit >> pt.text.get_text(dataset, 'text') >> reranker
  return model

model_EPIC = EPIC(dataset_trec,index_trec,limit)


In [ ]:
result_per_method,variante_sname = run(model_EPIC,"BM25+EPIC",query_variations)
df ,df_q = experiment(model_EPIC,result_per_method,query_variations,dataset_trec,variante_sname,"BM25+EPIC")
df

Methode :  WordSwapNeighboringCharacterSwap


<ipython-input-31-3849ac34831d>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_variation['query'] = query_variation['variation']


[2024-05-19 08:19:09,335][onir_pt][DEBUG] using GPU (deterministic)
[2024-05-19 08:19:09,490][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/1018 s<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/onir/rankers/epic.py:107: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:618.)
  result = torch.sparse.FloatTensor(torch.stack((idx0, idx1)), tok_salience.flatten(), s)


...

batches:   0%|          | 0/1052 s<?, ?it/s]

[2024-05-19 08:31:51,585][onir_pt][DEBUG] [finished] batches: [01:06] [1052it] [16.02it/s]
[2024-05-19 08:31:56,169][onir_pt][DEBUG] using GPU (deterministic)
[2024-05-19 08:31:56,175][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/1052 s<?, ?it/s]

[2024-05-19 08:33:01,792][onir_pt][DEBUG] [finished] batches: [01:06] [1052it] [16.03it/s]


,name,ndcg_cut_10,ndcg_cut_10 +,ndcg_cut_10 -,ndcg_cut_10 p-value
0,BM25+EPIC,0.624047,NaN,NaN,NaN
1,BM25+EPIC+WordSwapNeighboringCharacterSwap,0.389341,7.0,35.0,2.035201e-05
2,BM25+EPIC+WordSwapQWERTY,0.349635,6.0,33.0,9.957397e-07
3,BM25+EPIC+WordSwapRandomCharacterSubstitution,0.294983,5.0,36.0,9.891268e-08
4,BM25+EPIC+naturality_by_removing_stop_words,0.621385,14.0,18.0,8.372288e-01
5,BM25+EPIC+summarization_with_t5-base_from_desc...,0.506071,12.0,28.0,1.112595e-03
6,BM25+EPIC+WordInnerSwapRandom,0.622721,4.0,4.0,4.569341e-01
7,BM25+EPIC+back_translation_pivot_language_de,0.530082,9.0,19.0,7.479933e-03
8,BM25+EPIC+ramsrigouthamg/t5_paraphraser,0.604045,6.0,20.0,6.695272e-02
9,BM25+EPIC+WordSwapWordNet,0.449044,6.0,34.0,3.124799e-05


In [ ]:
# Chemin où vous souhaitez enregistrer le fichier CSV dans Google Drive
file_path = '/content/drive/My Drive/RI/BM25_EPIC.csv'

# Enregistrer le DataFrame en tant que fichier CSV
df.to_csv(file_path, index=False)
# Chemin où vous souhaitez enregistrer le fichier CSV dans Google Drive
file_path = '/content/drive/My Drive/RI/BM25_EPIC_q.csv'

# Enregistrer le DataFrame en tant que fichier CSV
df_q.to_csv(file_path, index=False)

#### BM25 CKNRM :

In [50]:

def CKNRM(dataset,index,limit):
  reranker = onir_pt.reranker.from_checkpoint("https://macavaney.us/pt_models/msmarco.convknrm.seed42.tar.gz")
  model = pt.BatchRetrieve(index, wmodel="BM25") % limit >> pt.text.get_text(dataset, 'text') >> reranker
  return model

model_CKNRM = CKNRM(dataset_trec,index_trec,limit)


In [ ]:
result_per_method,variante_sname = run(model_CKNRM,"BM25+CKNRM",query_variations)
df,df_q = experiment(model_CKNRM,result_per_method,query_variations,dataset_trec,variante_sname,"BM25+CKNRM")
df

Methode :  WordSwapNeighboringCharacterSwap


<ipython-input-31-3849ac34831d>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_variation['query'] = query_variation['variation']


[2024-05-19 08:03:17,676][onir_pt][DEBUG] using GPU (deterministic)
[2024-05-19 08:03:18,025][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/509 s<?, ?it/s]

[2024-05-19 08:03:33,950][onir_pt][DEBUG] [finished] batches: [15.92s] [509it] [31.97it/s]
BM25+CKNRM+WordSwapNeighboringCharacterSwap
Methode :  WordSwapQWERTY


...

batches:   0%|          | 0/526 s<?, ?it/s]

[2024-05-19 08:06:39,054][onir_pt][DEBUG] [finished] batches: [15.69s] [526it] [33.52it/s]
[2024-05-19 08:06:42,509][onir_pt][DEBUG] using GPU (deterministic)
[2024-05-19 08:06:42,511][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/526 s<?, ?it/s]

[2024-05-19 08:06:58,310][onir_pt][DEBUG] [finished] batches: [15.80s] [526it] [33.29it/s]


,name,ndcg_cut_10,ndcg_cut_10 +,ndcg_cut_10 -,ndcg_cut_10 p-value
0,BM25+CKNRM,0.493117,NaN,NaN,NaN
1,BM25+CKNRM+WordSwapNeighboringCharacterSwap,0.307987,14.0,27.0,1.453411e-03
2,BM25+CKNRM+WordSwapQWERTY,0.296539,9.0,30.0,1.017120e-04
3,BM25+CKNRM+WordSwapRandomCharacterSubstitution,0.226251,6.0,35.0,9.939431e-07
4,BM25+CKNRM+naturality_by_removing_stop_words,0.475599,18.0,17.0,3.847930e-01
5,BM25+CKNRM+summarization_with_t5-base_from_des...,0.392752,16.0,23.0,2.204821e-02
6,BM25+CKNRM+WordInnerSwapRandom,0.470756,13.0,26.0,1.073100e-01
7,BM25+CKNRM+back_translation_pivot_language_de,0.360486,8.0,23.0,3.210665e-03
8,BM25+CKNRM+ramsrigouthamg/t5_paraphraser,0.460877,11.0,16.0,6.399313e-02
9,BM25+CKNRM+WordSwapWordNet,0.360497,11.0,28.0,1.258938e-04


In [ ]:
# Chemin où vous souhaitez enregistrer le fichier CSV dans Google Drive
file_path = '/content/drive/My Drive/RI/BM25_CKNRM.csv'

# Enregistrer le DataFrame en tant que fichier CSV
df.to_csv(file_path, index=False)
# Chemin où vous souhaitez enregistrer le fichier CSV dans Google Drive
file_path = '/content/drive/My Drive/RI/BM25_CKNRM_q.csv'

# Enregistrer le DataFrame en tant que fichier CSV
df_q.to_csv(file_path, index=False)

## Experiment with originals before re ranking :

In [54]:
def run_with_original_for_bm25(dataset,index,model_name,query_variations,limit,reranker):
  variante_sname = []
  result_per_method = []
  #pre traitement
  query_variations['qid'] = query_variations['q_id'].astype(str)
  query_variations['query'] = query_variations['original_query']

  for method in query_variations['method'].unique():
      print("Methode : ",method)
      query_variation = query_variations[query_variations['method'] == method]
      query_variation['query'] = query_variation['variation']

      q_to_var = query_variation.set_index("query").to_dict()['variation']

      first_bm25 = pt.BatchRetrieve(index, wmodel="BM25") % limit

      rerank = pt.text.get_text(dataset, 'text') >> reranker

      first_res = first_bm25.transform(query_variations[['query','qid']].drop_duplicates()) # Les requetes originales

      first_res["query"] = first_res.apply(lambda r, d=q_to_var: d[r['query']] if r['query'] in d else r['query'], axis=1) # On lui passe les variations

      final_res = rerank.transform(first_res)

      result_per_method.append(final_res)
      variante_sname.append("{}+{}".format(model_name  , method))
      print("{}+{}".format(model_name, method))

  return result_per_method,variante_sname


def experiment(model,result_per_method,query_variations,dataset_trec,variante_sname,model_to_use):
  metrics = ['ndcg_cut_10']

  # Vérifier si first_res contient des valeurs None
  if query_variations.isnull().values.any():
        print("Warning: query_variations contains None values")
        print(query_variations[query_variations.isnull().any(axis=1)])

  df = pt.Experiment(
        [model] + result_per_method,
        query_variations[['qid','query']].drop_duplicates(),
        dataset_trec.get_qrels(),
        metrics,
        baseline=0,
        names = [model_to_use]+variante_sname)
  df_perq = pt.Experiment(
        [model] + result_per_method,
        query_variations[['query','qid']].drop_duplicates(),
        dataset_trec.get_qrels(),
        metrics,
        perquery = True,
        names = [model_to_use]+variante_sname)
  return df,df_perq


bm_25_on_original = pt.BatchRetrieve(index_trec, wmodel="BM25") % limit

#### bm25:

#### Bert :

In [21]:
bert = onir_pt.reranker.from_checkpoint("https://macavaney.us/scibert-medmarco.tar.gz",expected_md5="854966d0b61543ffffa44cea627ab63b")



[2024-05-19 10:27:17,974][onir_pt][INFO] using cached checkpoint: /root/data/onir/model_checkpoints/1153203ea2abbdf7f760fa6956a83d83


In [22]:
result_per_method,variante_sname = run_with_original_for_bm25(dataset_trec,index_trec,"BM25+onoriginal+BERT",query_variations,limit,bert)

pipeline = bm_25_on_original >> pt.text.get_text(dataset_trec, 'text') >> bert
df,df_q = experiment(pipeline,result_per_method,query_variations,dataset_trec,variante_sname,"BM25+onoriginal+BERT")
df


Methode :  WordSwapNeighboringCharacterSwap


<ipython-input-16-a87fa34d9d14>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_variation['query'] = query_variation['variation']


[2024-05-19 10:28:00,026][onir_pt][DEBUG] using GPU (deterministic)
[2024-05-19 10:28:00,186][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/1052 s<?, ?it/s]

[2024-05-19 10:28:37,461][onir_pt][DEBUG] [finished] batches: [37.27s] [1052it] [28.22it/s]
BM25+onoriginal+BERT+WordSwapNeighboringCharacterSwap
Methode :  WordSwapQWERTY


...

batches:   0%|          | 0/1052 s<?, ?it/s]

[2024-05-19 10:35:16,273][onir_pt][DEBUG] [finished] batches: [36.41s] [1052it] [28.90it/s]
[2024-05-19 10:35:21,053][onir_pt][DEBUG] using GPU (deterministic)
[2024-05-19 10:35:21,057][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/1052 s<?, ?it/s]

[2024-05-19 10:35:57,256][onir_pt][DEBUG] [finished] batches: [36.20s] [1052it] [29.06it/s]


,name,ndcg_cut_10,ndcg_cut_10 +,ndcg_cut_10 -,ndcg_cut_10 p-value
0,BM25+onoriginal+BERT,0.628664,NaN,NaN,NaN
1,BM25+onoriginal+BERT+WordSwapNeighboringCharac...,0.628664,0.0,0.0,NaN
2,BM25+onoriginal+BERT+WordSwapQWERTY,0.628664,0.0,0.0,NaN
3,BM25+onoriginal+BERT+WordSwapRandomCharacterSu...,0.628664,0.0,0.0,NaN
4,BM25+onoriginal+BERT+naturality_by_removing_st...,0.628664,0.0,0.0,NaN
5,BM25+onoriginal+BERT+summarization_with_t5-bas...,0.628664,0.0,0.0,NaN
6,BM25+onoriginal+BERT+WordInnerSwapRandom,0.628664,0.0,0.0,NaN
7,BM25+onoriginal+BERT+back_translation_pivot_la...,0.628664,0.0,0.0,NaN
8,BM25+onoriginal+BERT+ramsrigouthamg/t5_paraphr...,0.628664,0.0,0.0,NaN
9,BM25+onoriginal+BERT+WordSwapWordNet,0.628664,0.0,0.0,NaN


In [58]:
df_q

,name,qid,measure,value
0,BM25+onoriginal+BERT,1037798,ndcg_cut_10,0.251049
1,BM25+onoriginal+BERT,104861,ndcg_cut_10,0.706544
2,BM25+onoriginal+BERT,1063750,ndcg_cut_10,0.411664
3,BM25+onoriginal+BERT,1103812,ndcg_cut_10,0.807207
4,BM25+onoriginal+BERT,1106007,ndcg_cut_10,0.527770
...,...,...,...,...
253,BM25+onoriginal+BERT+summarization_with_t5-bas...,855410,ndcg_cut_10,0.990661
254,BM25+onoriginal+BERT+summarization_with_t5-bas...,87181,ndcg_cut_10,0.436212
255,BM25+onoriginal+BERT+summarization_with_t5-bas...,87452,ndcg_cut_10,0.498365
256,BM25+onoriginal+BERT+summarization_with_t5-bas...,915593,ndcg_cut_10,0.436838


In [57]:
df

,name,ndcg_cut_10,ndcg_cut_10 +,ndcg_cut_10 -,ndcg_cut_10 p-value
0,BM25+onoriginal+BERT,0.628664,NaN,NaN,NaN
1,BM25+onoriginal+BERT+WordSwapNeighboringCharac...,0.628664,0.0,0.0,NaN
2,BM25+onoriginal+BERT+WordSwapQWERTY,0.628664,0.0,0.0,NaN
3,BM25+onoriginal+BERT+WordSwapRandomCharacterSu...,0.628664,0.0,0.0,NaN
4,BM25+onoriginal+BERT+naturality_by_removing_st...,0.628664,0.0,0.0,NaN
5,BM25+onoriginal+BERT+summarization_with_t5-bas...,0.628664,0.0,0.0,NaN
6,BM25+onoriginal+BERT+WordInnerSwapRandom,0.628664,0.0,0.0,NaN
7,BM25+onoriginal+BERT+back_translation_pivot_la...,0.628664,0.0,0.0,NaN
8,BM25+onoriginal+BERT+ramsrigouthamg/t5_paraphr...,0.628664,0.0,0.0,NaN
9,BM25+onoriginal+BERT+WordSwapWordNet,0.628664,0.0,0.0,NaN


In [62]:
# Chemin où vous souhaitez enregistrer le fichier CSV dans Google Drive
file_path = '/content/drive/My Drive/RI/BM25_original_bert.csv'

# Enregistrer le DataFrame en tant que fichier CSV
df.to_csv(file_path, index=False)
# Chemin où vous souhaitez enregistrer le fichier CSV dans Google Drive
file_path = '/content/drive/My Drive/RI/BM25_original_bert_q.csv'

# Enregistrer le DataFrame en tant que fichier CSV
df_q.to_csv(file_path, index=False)

#### KNRM :

In [24]:
knrm = onir_pt.reranker.from_checkpoint('https://macavaney.us/knrm.medmarco.tar.gz',  expected_md5="d70b1d4f899690dae51161537e69ed5a")


[2024-05-19 10:44:59,309][onir.util.download][DEBUG] downloaded https://macavaney.us/knrm.medmarco.tar.gz s] [1.43k] [1.74MB/s] [md5 hash verified]
[2024-05-19 10:44:59,324][WordvecHashVocab][DEBUG] [starting] downloading https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip


[2024-05-19 10:45:06,823][onir.util.download][WARNING] no hash provided for https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip; consider adding expected_md5="3cc8839ac3fa9a6187149b1e73328b2a" to ensure data integrity.
[2024-05-19 10:45:06,830][onir.util.download][DEBUG] downloaded https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip [7.37s] [682M] [114MB/s]
[2024-05-19 10:45:06,845][WordvecHashVocab][DEBUG] [finished] downloading https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip [7.52s]
[2024-05-19 10:45:06,845][WordvecHashVocab][DEBUG] [starting] extracting vecs
[2024-05-19 10:45:33,536][WordvecHashVocab][DEBUG] [finished] extracting vecs [26.69s]
[2024-05-19 10:45:33,537][WordvecHashVocab][DEBUG] [starting] loading vecs into memory
[2024-05-19 10:48:16,047][WordvecHashVocab][DEBUG] [finished] loading vecs into memory [02:43]
[2024-05-19 10:48:16,263][WordvecHashVocab][DEBUG] [starting] writing

In [33]:
pipeline=  pt.BatchRetrieve(index_trec, wmodel="BM25") % limit >> knrm


result_per_method2,variante_sname2 = run_with_original_for_bm25(dataset_trec,index_trec,"BM25+onoriginal+KNRM",query_variations,limit,knrm)

df2,df_q2 = experiment(pipeline,result_per_method2,query_variations,dataset_trec,variante_sname2,"BM25+onoriginal+KNRM")
df2

Methode :  WordSwapNeighboringCharacterSwap


<ipython-input-32-a87fa34d9d14>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_variation['query'] = query_variation['variation']


[2024-05-19 11:04:02,678][onir_pt][DEBUG] using GPU (deterministic)
[2024-05-19 11:04:02,679][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/1052 s<?, ?it/s]

[2024-05-19 11:04:04,361][onir_pt][DEBUG] [finished] batches: [1.68s] [1052it] [625.73it/s]
BM25+onoriginal+KNRM+WordSwapNeighboringCharacterSwap
Methode :  WordSwapQWERTY


...

[2024-05-19 11:04:53,228][onir_pt][DEBUG] using GPU (deterministic)
[2024-05-19 11:04:53,230][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/1052 s<?, ?it/s]

[2024-05-19 11:04:54,894][onir_pt][DEBUG] [finished] batches: [1.66s] [1052it] [632.37it/s]
BM25+onoriginal+KNRM+WordSwapEmbedding
[2024-05-19 11:04:58,114][onir_pt][DEBUG] using GPU (deterministic)
[2024-05-19 11:04:58,115][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/1052 s<?, ?it/s]

TypeError: object of type 'NoneType' has no len()

In [47]:
df_q2 = experiment(pipeline,result_per_method2,query_variations,dataset_trec,variante_sname2,"BM25+onoriginal+KNRM")
df_q2

[2024-05-19 11:16:11,272][onir_pt][DEBUG] using GPU (deterministic)
[2024-05-19 11:16:11,274][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/1052 s<?, ?it/s]

TypeError: object of type 'NoneType' has no len()

#### CONVKNRM :

In [51]:
model_CKNRM = CKNRM(dataset_trec,index_trec,limit)


[2024-05-19 11:19:06,630][onir.util.download][WARNING] no hash provided for https://macavaney.us/pt_models/msmarco.convknrm.seed42.tar.gz; consider adding expected_md5="6e57b8d728151debcb23479c8a41f4b0" to ensure data integrity.
[2024-05-19 11:19:06,637][onir.util.download][DEBUG] downloaded https://macavaney.us/pt_models/msmarco.convknrm.seed42.tar.gz s] [860k] [1.88MB/s]
[2024-05-19 11:19:06,661][WordvecHashVocab][DEBUG] [starting] reading cached at /root/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p
[2024-05-19 11:19:17,999][WordvecHashVocab][DEBUG] [finished] reading cached at /root/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p [11.34s]


In [60]:
pipeline=  pt.BatchRetrieve(index_trec, wmodel="BM25") % limit >> pt.text.get_text(dataset_trec, 'text') >> model_CKNRM


result_per_method2,variante_sname2 = run_with_original_for_bm25(dataset_trec,index_trec,"BM25+onoriginal+CONVKNRM",query_variations,limit,model_CKNRM)

df2,df_q2 = experiment(pipeline,result_per_method2,query_variations,dataset_trec,variante_sname2,"BM25+onoriginal+CONVKNRM")
df2

Methode :  WordSwapNeighboringCharacterSwap


<ipython-input-54-5865c6a9448f>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_variation['query'] = query_variation['variation']


[2024-05-19 11:38:29,790][onir_pt][DEBUG] using GPU (deterministic)
[2024-05-19 11:38:29,792][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/526 s<?, ?it/s]

[2024-05-19 11:38:45,124][onir_pt][DEBUG] [finished] batches: [15.33s] [526it] [34.31it/s]
BM25+onoriginal+CONVKNRM+WordSwapNeighboringCharacterSwap
Methode :  WordSwapQWERTY


...

batches:   0%|          | 0/526 s<?, ?it/s]

[2024-05-19 11:42:09,248][onir_pt][DEBUG] [finished] batches: [16.13s] [526it] [32.62it/s]
[2024-05-19 11:42:14,961][onir_pt][DEBUG] using GPU (deterministic)
[2024-05-19 11:42:14,963][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/526 s<?, ?it/s]

[2024-05-19 11:42:30,445][onir_pt][DEBUG] [finished] batches: [15.48s] [526it] [33.98it/s]


,name,ndcg_cut_10,ndcg_cut_10 +,ndcg_cut_10 -,ndcg_cut_10 p-value
0,BM25+onoriginal+CONVKNRM,0.493117,NaN,NaN,NaN
1,BM25+onoriginal+CONVKNRM+WordSwapNeighboringCh...,0.493117,0.0,0.0,NaN
2,BM25+onoriginal+CONVKNRM+WordSwapQWERTY,0.493117,0.0,0.0,NaN
3,BM25+onoriginal+CONVKNRM+WordSwapRandomCharact...,0.493117,0.0,0.0,NaN
4,BM25+onoriginal+CONVKNRM+naturality_by_removin...,0.493117,0.0,0.0,NaN
5,BM25+onoriginal+CONVKNRM+summarization_with_t5...,0.493117,0.0,0.0,NaN
6,BM25+onoriginal+CONVKNRM+WordInnerSwapRandom,0.493117,0.0,0.0,NaN
7,BM25+onoriginal+CONVKNRM+back_translation_pivo...,0.493117,0.0,0.0,NaN
8,BM25+onoriginal+CONVKNRM+ramsrigouthamg/t5_par...,0.493117,0.0,0.0,NaN
9,BM25+onoriginal+CONVKNRM+WordSwapWordNet,0.493117,0.0,0.0,NaN


In [61]:
df_q2

,name,qid,measure,value
0,BM25+onoriginal+CONVKNRM,1037798,ndcg_cut_10,0.088377
1,BM25+onoriginal+CONVKNRM,104861,ndcg_cut_10,0.382716
2,BM25+onoriginal+CONVKNRM,1063750,ndcg_cut_10,0.053669
3,BM25+onoriginal+CONVKNRM,1103812,ndcg_cut_10,0.693218
4,BM25+onoriginal+CONVKNRM,1106007,ndcg_cut_10,0.584736
...,...,...,...,...
253,BM25+onoriginal+CONVKNRM+summarization_with_t5...,855410,ndcg_cut_10,0.881199
254,BM25+onoriginal+CONVKNRM+summarization_with_t5...,87181,ndcg_cut_10,0.372330
255,BM25+onoriginal+CONVKNRM+summarization_with_t5...,87452,ndcg_cut_10,0.209094
256,BM25+onoriginal+CONVKNRM+summarization_with_t5...,915593,ndcg_cut_10,0.331937


In [63]:
# Chemin où vous souhaitez enregistrer le fichier CSV dans Google Drive
file_path = '/content/drive/My Drive/RI/BM25_original_CONVKNRM.csv'

# Enregistrer le DataFrame en tant que fichier CSV
df2.to_csv(file_path, index=False)
# Chemin où vous souhaitez enregistrer le fichier CSV dans Google Drive
file_path = '/content/drive/My Drive/RI/BM25_original_CONVKNRM_q.csv'

# Enregistrer le DataFrame en tant que fichier CSV
df_q2.to_csv(file_path, index=False)

#### EPIC:

In [68]:
model_EPIC = EPIC(dataset_trec,index_trec,limit)


[2024-05-19 11:50:56,792][onir.util.download][WARNING] no hash provided for https://macavaney.us/pt_models/msmarco.epic.seed42.tar.gz; consider adding expected_md5="c184a924debf63b5115e34a3b92377e0" to ensure data integrity.
[2024-05-19 11:50:56,804][onir.util.download][DEBUG] downloaded https://macavaney.us/pt_models/msmarco.epic.seed42.tar.gz [16.00s] [495M] [17.8MB/s]


100%|██████████| 231508/231508 s<0ms, 1147198.19B/s] 
100%|██████████| 433/433 s<0ms, 318173.38B/s]
100%|██████████| 440473133/440473133 [7.74s<0ms, 56915214.57B/s]


In [69]:
pipeline=  pt.BatchRetrieve(index_trec, wmodel="BM25") % limit >> pt.text.get_text(dataset_trec, 'text') >> model_EPIC


result_per_method2,variante_sname2 = run_with_original_for_bm25(dataset_trec,index_trec,"BM25+onoriginal+EPIC",query_variations,limit,model_EPIC)

df3,df_q3 = experiment2(pipeline,result_per_method2,query_variations,dataset_trec,variante_sname2,"BM25+onoriginal+EPIC")
df3

Methode :  WordSwapNeighboringCharacterSwap


<ipython-input-54-5865c6a9448f>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_variation['query'] = query_variation['variation']


[2024-05-19 11:53:13,577][onir_pt][DEBUG] using GPU (deterministic)
[2024-05-19 11:53:13,756][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/1052 s<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/onir/rankers/epic.py:107: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:618.)
  result = torch.sparse.FloatTensor(torch.stack((idx0, idx1)), tok_salience.flatten(), s)


...

batches:   0%|          | 0/1052 s<?, ?it/s]

[2024-05-19 12:05:24,395][onir_pt][DEBUG] [finished] batches: [01:02] [1052it] [16.94it/s]
[2024-05-19 12:05:28,757][onir_pt][DEBUG] using GPU (deterministic)
[2024-05-19 12:05:28,761][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/1052 s<?, ?it/s]

[2024-05-19 12:06:30,123][onir_pt][DEBUG] [finished] batches: [01:01] [1052it] [17.14it/s]


,name,ndcg_cut_10,ndcg_cut_10 +,ndcg_cut_10 -,ndcg_cut_10 p-value
0,BM25+onoriginal+EPIC,0.624047,NaN,NaN,NaN
1,BM25+onoriginal+EPIC+WordSwapNeighboringCharac...,0.624047,0.0,0.0,NaN
2,BM25+onoriginal+EPIC+WordSwapQWERTY,0.624047,0.0,0.0,NaN
3,BM25+onoriginal+EPIC+WordSwapRandomCharacterSu...,0.624047,0.0,0.0,NaN
4,BM25+onoriginal+EPIC+naturality_by_removing_st...,0.624047,0.0,0.0,NaN
5,BM25+onoriginal+EPIC+summarization_with_t5-bas...,0.624047,0.0,0.0,NaN
6,BM25+onoriginal+EPIC+WordInnerSwapRandom,0.624047,0.0,0.0,NaN
7,BM25+onoriginal+EPIC+back_translation_pivot_la...,0.624047,0.0,0.0,NaN
8,BM25+onoriginal+EPIC+ramsrigouthamg/t5_paraphr...,0.624047,0.0,0.0,NaN
9,BM25+onoriginal+EPIC+WordSwapWordNet,0.624047,0.0,0.0,NaN


In [70]:
df_q3

,name,qid,measure,value
0,BM25+onoriginal+EPIC,1037798,ndcg_cut_10,0.373656
1,BM25+onoriginal+EPIC,104861,ndcg_cut_10,0.309352
2,BM25+onoriginal+EPIC,1063750,ndcg_cut_10,0.324037
3,BM25+onoriginal+EPIC,1103812,ndcg_cut_10,0.781462
4,BM25+onoriginal+EPIC,1106007,ndcg_cut_10,0.792126
...,...,...,...,...
253,BM25+onoriginal+EPIC+summarization_with_t5-bas...,855410,ndcg_cut_10,0.990661
254,BM25+onoriginal+EPIC+summarization_with_t5-bas...,87181,ndcg_cut_10,0.740639
255,BM25+onoriginal+EPIC+summarization_with_t5-bas...,87452,ndcg_cut_10,0.438106
256,BM25+onoriginal+EPIC+summarization_with_t5-bas...,915593,ndcg_cut_10,0.358954


In [71]:
# Chemin où vous souhaitez enregistrer le fichier CSV dans Google Drive
file_path = '/content/drive/My Drive/RI/BM25_original_EPIC.csv'

# Enregistrer le DataFrame en tant que fichier CSV
df3.to_csv(file_path, index=False)
# Chemin où vous souhaitez enregistrer le fichier CSV dans Google Drive
file_path = '/content/drive/My Drive/RI/BM25_original_EPIC_q.csv'

# Enregistrer le DataFrame en tant que fichier CSV
df_q3.to_csv(file_path, index=False)